# Создадим предобученную модель

In [1]:
from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")

/home/admin-gpu/.cache/pypoetry/virtualenvs/homework5-segmentation-u-9vj8JT-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]          

## Измерим параметры на ГПУ

In [2]:
# Validate the model
# При валлидации происходит Fusing слоев модели!
# было 261 слой, стало 195 слоев ! 
# уменьшение слоев примерно на 25%
metrics = model.val(data="ms_coco_val_2017.yaml", device=0)

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework1/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:25<00:00, 12.43it/s]
                   all       5000      36335      0.692      0.555      0.609      0.444      0.683      0.537       0.58      0.371
                person       5000      10777      0.801      0.712      0.797      0.575      0.796      0.694      0.774      0.465
               bicycle       5000        314       0.75      0.481      0.573      0.337      0.665      0.411      0.462      0.195
                   car       5000       1918     

## Измерим параметры на ЦПУ

In [3]:
metrics = model.val(data="ms_coco_val_2017.yaml", device='cpu')

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework1/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [07:45<00:00,  1.49s/it]
                   all       5000      36335      0.692      0.555      0.609      0.445      0.684      0.536       0.58      0.371
                person       5000      10777      0.801      0.712      0.797      0.575      0.798      0.694      0.774      0.465
               bicycle       5000        314      0.751      0.481      0.573      0.337      0.666      0.411      0.462      0.194
                   car       5000       1918      0.717      0.588      0.652      0.443      0.704      0.564    

In [4]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.371
mask map50 0.58
mask map75 0.396


In [5]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.pt") / 1024**2
print(f"{model_size:.3} Мб")

22.8 Мб


# Экспорт моделей в различные форматы

Согласно официальной документации и туториалу `YOLOv8` поддерживает экспорт в множество различных форматов. 

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`                                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`                                    |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [ncnn](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


# Экспорт в ONNX

In [6]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [7]:
model.export(format="onnx")

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)

ONNX: starting export with onnx 1.14.1 opset 16...
ONNX: export success ✅ 0.6s, saved as 'yolov8s-seg.onnx' (45.3 MB)

Export complete (2.0s)
Results saved to /home/admin-gpu/Downloads/yolo_VIKA/homework5/segmentation
Predict:         yolo predict task=segment model=yolov8s-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=yolov8s-seg.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s-seg.onnx'

In [8]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

## Измерим параметры на ГПУ

In [9]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CUDA:0 (NVIDIA A10, 24074MiB)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework1/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [09:48<00:00,  8.50it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.462       0

## Измерим параметры на ЦПУ

In [10]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading yolov8s-seg.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /home/admin-gpu/Downloads/yolo_VIKA/homework1/segmentation/datasets/ms_coco_val_2017/val/labels.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5000/5000 [08:38<00:00,  9.64it/s]
                   all       5000      36335      0.692      0.552      0.606      0.444      0.686      0.531      0.577       0.37
                person       5000      10777      0.798      0.711      0.796      0.574      0.797      0.691      0.772      0.464
               bicycle       5000        314      0.728      0.465      0.567      0.335      0.644      0.398      0.46

In [11]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

mask map50-95 0.37
mask map50 0.577
mask map75 0.395


In [12]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")

45.3 Мб


# Экспорт в TensorRT

In [1]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")

/home/admin-gpu/.cache/pypoetry/virtualenvs/homework5-segmentation-u-9vj8JT-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]          

In [2]:
model.export(format="engine")

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-1.13.1+cu117 CUDA:0 (NVIDIA A10, 24074MiB)
YOLOv8s-seg summary (fused): 195 layers, 11810560 parameters, 0 gradients

PyTorch: starting from 'yolov8s-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (22.8 MB)
requirements: Ultralytics requirement ['nvidia-tensorrt'] not found, attempting AutoUpdate...
TensorRT: export failure ❌ 7.5s: No module named 'tensorrt'


ModuleNotFoundError: No module named 'tensorrt'

ERROR: Could not install packages due to an OSError: [Errno 32] Broken pipe



In [ ]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

## Измерим параметры на ГПУ

In [ ]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device=0)

In [ ]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

In [ ]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")

# Экспорт в OpenVino

In [ ]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# Load a model
# build a new model from scratch
model = YOLO("yolov8s-seg.yaml")
# load a pretrained model (recommended for training)
model = YOLO("yolov8s-seg.pt")

In [ ]:
model.export(format="openvino")

In [ ]:
# Load a model
model = YOLO("yolov8s-seg.onnx")

## Измерим параметры на ЦПУ

In [ ]:
metrics = model.val(data="ms_coco_val_2017.yaml", imgsz=640, device='cpu')

In [ ]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.seg.map:.3}")
print(f"mask map50 {metrics.seg.map50:.3}")
print(f"mask map75 {metrics.seg.map75:.3}")

In [ ]:
# оценим размер сериализованной модели
model_size = os.path.getsize("yolov8s-seg.onnx") / 1024**2
print(f"{model_size:.3} Мб")